### conv gray

In [190]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2
import os
import glob


def convolve_2d(image, kernel):
    (ih, iw) = image.shape
    (kh, kw) = kernel.shape

    # "pad" the borders of the input image
    pad = (kh-1)//2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((ih, iw), dtype="float32")

    # loop over the input image, "sliding" the kernel across
    # each (x, y)-coordinate from left-to-right and top to
    # bottom
    for y in np.arange(pad, ih + pad):
         for x in np.arange(pad, iw + pad):
            # extract the ROI of the image by extracting the
            # *center* region of the current (x, y)-coordinates
            # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            # perform the actual convolution by taking the
            # element-wise multiplicate between the ROI and
            # the kernel, then summing the matrix
            k = (roi * kernel).sum()
            # store the convolved value in the output (x,y)-
            # coordinate of the output image
            output[y - pad, x - pad] = k

    # rescale the output image to be in the range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
    # return the output image
    return output

In [191]:
# construct a sharpening filter
sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]), dtype="int")


image = cv2.imread(r"E:\AI Practice Project\cv\basic project\2-Face detection with OpenCV-ssd-caffe\group2.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

convoleOutput_2d = convolve_2d(gray, kernel= sharpen)


cv2.imshow('conv', convoleOutput_2d)
cv2.waitKey(0)
cv2.destroyAllWindows()


### conv rgb

In [200]:
import numpy as np
import cv2
from PIL import Image


'''
I would like to give an insight into my approach :

1-The red, blue, and green pixels are separated into different arrays.
2-Employ the process of convolution individually to each of the streams.
3-This results in 3 arrays to which convolution has been applied.
4-Combine the three arrays and convert the RGB values into an image.

'''

image = cv2.imread(r"E:\AI Practice Project\cv\basic project\2-Face detection with OpenCV-ssd-caffe\group2.jpg")
height = image.shape[0]
weight = image.shape[1]

b = image[:,:,0]
g = image[:,:,1]
r = image[:,:,2]


# blur kernel
kernel = np.array([0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11]).reshape(3,3)

def convolute(image, height, weight, kernel):
    convoluted_matrix = []
    for i in range(1, height-2):
        for j in range(1, weight-2):
            roi = image[i:i+3, j:j+3]
            prod = np.multiply(roi, kernel)
            convoluted_matrix.append(np.sum(prod))

    convoluted_matrix = np.array(convoluted_matrix).reshape(height-3, weight-3)
    return convoluted_matrix


b_convol = convolute(b, height, weight, kernel)
g_convol = convolute(g, height, weight, kernel)
r_convol = convolute(r, height, weight, kernel)

height = height - 3
weight = weight - 3

combine = np.zeros([height, weight, 3], dtype=np.uint8)

for i in range(height):
    for j in range(weight):

        combine[i, j] = [int(b_convol[i,j]), int(g_convol[i,j]), int(r_convol[i,j])]

#from PIL import Image
#img = Image.fromarray(combine)





In [208]:

cv2.imshow('conv_rgb_blur', combine)
cv2.imshow('original', image)

cv2.waitKey(0)
cv2.destroyAllWindows()
